# Exercise 1: Getting Started with Azure ML

Azure Machine Learning (*Azure ML*) is a cloud-based service for creating and managing machine learning solutions. It's designed to help data scientists leverage their existing data processing and model development skills and frameworks, and help them scale their workloads to the cloud.

## Install the Azure ML SDK for Python

The Azure ML SDK for Python provides classes you can use to work with Azure ML in your Azure subscription. The SDK is pre-installed in the Azure ML notebooks environment, but it's worth checking that you have the latest version of the package installed. Run the cell below to install the **azureml-sdk** Python package, including the optional *notebooks* component; which provides some functionality specific to the Jupyter Notebooks environment.

> **More Information**: For more details about installing the Azure ML SDK and its optional components, see the [Azure ML SDK Documentation](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)

In [ ]:
!pip install --upgrade azureml-sdk[notebooks]

import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

## Connect to Your Workspace

All experiments and associated resources are managed within you Azure ML workspace. You can connect to an existing workspace, or create a new one using the Azure ML SDK.

In most cases, you should store the workspace configuration in a JSON configuration file. This makes it easier to reconnect without needing to remember details like your Azure subscription ID. You can download the JSON configuration file from the blade for your workspace in the Azure portal, but if you're using a Notebook VM within your wokspace, the configuration file has alreday been downloaded to the root folder.

The code below uses the configuration file to connect to your workspace. The first time you run it in a notebook session, you may be prompted to sign into Azure by clicking the https://microsoft.com/devicelogin link,  entering an automatically generated code, and signing into Azure. After you have successfully signed in, you can close the browser tab that was opened and return to this notebook.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

**Note**: If you are running code in an environment other than an Azure ML Notebook VM, you can either download the JSON configuration file from the Azure portal to the folder containing your code, or you can explicitly connect to a workspace by specifying the workspace name, subscription ID, and resource group; and then save the configuration for reuse later like this:

```Python
from azureml.core import Workspace

ws = Workspace(workspace_name=WORKSPACE_NAME,
               subscription_id=SUBSCRIPTION_ID,
               resource_group= RESOURCE_GROUP_NAME)

ws.write_config()
```

## Manage Azure ML Assets

Now that you have a connection to your workspace, you can view the assets it contains. For example, to see the compute available in your workspace, run the cell below:

In [ ]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("Compute Targets:")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ":", compute.type)

In Azure ML, *datastores* are references to storage locations, such as Azure Storage blob containers, Azure Data Lake storage, Azure SQL Database instances, and so on. Every workspace has a *default* datastore - usually the Azure storage blob container that was created with the workspace. If you need to work with data that is stored in different locations, you can add custom datastores to your workspace and set any of them to be the default.

Run the following code to determine the datastores in your workspace:

In [ ]:
from azureml.core import Datastore

# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

You can also view and manage datastores in your workspace on the **Datastores** page for your workspace in the [Azure ML Studio Web interface](https://ml.azure.com).

Now that you have determined the available datastores, you can upload files from your local file system to a datastore so that it will be accessible to experiments running in the workspace, regardless of where the experiment script is actually being run.

In [ ]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Note that the upload to the datastore results in the creation of a *data reference*, which is an abstraction that represents the connection to the data in the datastore.

So now there's a copy of the diabetes data in the default datastore for the workspace, that you can use in future experiments. You *can* read the data files directly from the datastore (using a data reference), but if you want to take advantage of Azure ML's ability to track multiple versions of training data or detect data drift, you need to get familiar with another data-related object in Azure ML - the *dataset*.

A dataset is an object that encapsulates a specific data source. Let's create a dataset from the diabetes data you uploaded to the datastore, and view the first 20 records. In this case, the data is in a structured format in a CSV file, so we'll use a *Tabular* dataset.

In [ ]:
from azureml.core import Dataset

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

Now that you have a dataset that references the diabetes data, you can register it to make it easily accessible to any experiment being run in the workspace.

In [ ]:
# Register the dataset
dataset_name = 'diabetes dataset'
tab_data_set = tab_data_set.register(workspace=ws, 
                           name=dataset_name,
                           description='diabetes data',
                           tags = {'year':'2019', 'category':'Diabetes'},
                           create_new_version=True)

# List the datasets registered in the workspace
for ds in ws.datasets:
    print(ws.datasets[ds].name)

You can also view and manage datasets on the **Datasets** page for your workspace in the [Azure ML Studio web interface](https://ml.azure.com).

> **Note**: The dataset you've created is a *tabular* dataset, and can be read as a dataframe containing all of the data in the structured files that are included in the dataset definition. This works well for tabular data, but in some machine learning scenarios you'll need to work with data that is unstructured; or you may simply want to handle reading the data from files in your own code. To accomplish this, you can use a *file* dataset, which creates a list of file paths in a virtual mount point, which you can use to read the data in the files. For more information, see [the documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-register-datasets#dataset-types).


## Train a Model

OK, now we're ready to start training models. We'll use the diabetes data to train a classification model that predicts whether or not a patient is likely to be diabetic. This is a fairly simple example that can be accomplished easily using a statistical machine learning library like scikit-learn, but it will give us an opportunity to explore how Azure ML can be used to manage model training.

In Azure ML, you train models by running code as an *experiment*, from which you can log metrics for later analysis. First, let's create an experiment and a local folder into which we'll put the files needed to run it.

In [ ]:
import os
from azureml.core import Experiment

# Create an experiment
experiment_name = 'diabetes_training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Create a folder for the experiment files
experiment_folder = './' + experiment_name
os.makedirs(experiment_folder, exist_ok=True)

print("Experiment:", experiment.name)

Next, create a Python script file for the experiment (we're creating the script file dynamically here, but in production you'd likely create it separately and store it in a source control system).

If you're familiar with scikit-learn, this script shouldn't contain too many surprises for you. The key Azure ML-specific points to note are:

- The script uses the experiment run context to log metrics in your Azure ML workspace.
- The script obtains the training data from a dataset that is passed as an input to the run (you'll see how shortly!).

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Workspace, Dataset, Experiment, Run
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe() # Get the training data from the estimator input

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

# log a ROC curve plot
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Azure ML supports the use of an *Estimator* object that is designed specifically for model training. There's a generic estimator, and some framework specific estimators for common machine learning frameworks like Scikit-Learn, PyTorch, and TensorFlow.

> **More Information**: For more details about estimators, see the [Azure ML documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-ml-models).

Let's run the experiment to train our diabetes model using an estimator. We'll run it on *local* compute (that is, the Notebook VM) in a conda environment that includs all of the necessary Python packages.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Set the script parameters
script_params = {
    '--regularization': 0.1
}

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a Python environment for the experiment
environment_name = "diabetes-env"
diabetes_env = Environment(environment_name)
diabetes_env.python.user_managed_dependencies = False 
diabetes_env.docker.enabled = False
diabetes_conda = CondaDependencies.create(conda_packages=['pandas','scikit-learn','joblib','ipykernel','matplotlib'],
                                          pip_packages=['azureml-sdk','argparse','pyarrow'])
diabetes_env.python.conda_dependencies = diabetes_conda
# Register the environment so we can re-use it later.
diabetes_env.register(ws)

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the dataset as an input
              script_params=script_params,
              compute_target = 'local', # Use local compute
              environment_definition = Environment.get(ws, environment_name),
              entry_script='diabetes_training.py')

# Run the experiment
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

Now that the experiment has finished, we can take advantage of one of the widgets provided in the *notebooks* package we included when we installed the Azure ML SDK, and view the details of the run.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

You view more details, including charts and images based on the metrics and plots logged by the script, in the **Experiments** tab in [Azure ML Studio](https://ml.azure.com). Just open the **diabetes_training** experiment and view its most recent run.

The logged metrics and output files from the experiment run are also available through the SDK, like this:

In [ ]:
import json

print(json.dumps(run.get_metrics(), indent=2))
print(json.dumps(run.get_file_names(), indent=2))

The model we trained is saved as the **diabetes_model.pkl** file in the **outputs** folder. Since we've gone to the effort of creating it, let's register it in the workspace so we can easily retrieve it again later.

In [ ]:
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator (tabular dataset)'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Switch to the [Azure ML Studio web interface](https://ml.azure.com) and view the **Models** in your Azure ML workspace. The *diabetes_model* model should be listed, and clicking it reveals its details.

You can also examine the models in a workspace using code, like this:

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## Deploy the Model as a Web Service

Let's get the registered model that we want to deploy. By default, if we specify a model name, the latest version will be returned.

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

We're going to create a web service to host this model, and this will require some code and configuration files; so let's create a folder for those.

In [ ]:
import os

folder_name = 'diabetes_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

The web service where we deploy the model will need some Python code to load the input data, get the model from the workspace, and generate and return predictions. We'll save this code in a *entry script* file that will be deployed to the web service:

In [ ]:
%%writefile $folder_name/score_diabetes.py
import json
import numpy as np
import os
import joblib
from azureml.core.model import Model
import azureml.train.automl # Required for AutoML models with pre-processing

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data - the features of patients to be classified.
    data = json.loads(raw_data)['data']
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

The web service will be hosted in a container, and the container will need to install any required Python dependencies when it gets initialized. In this case, our scoring code requires **scikit-learn**, so we'll create a .yml file that tells the container host to install this into the environment.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Now you're ready to deploy. We'll deploy the container a service named **diabetes-service**. The deployment process includes the following steps:

1. Define an inference configuration, which includes the scoring and environment files required to load and use the model.
2. Define a deployment configuration that defines the execution environment in which the service will be hosted. In this case, an Azure Container Instance (ACI).
3. Deploy the model as a web service.
4. Verify the status of the deployed service.

> **Note**: ACI is useful for testing the deployment of a model as a web service. For production deployment, you should use an Azure Kubernetes Service (AKS) cluster for increased scalability and security. For more details about model deployment, and options for target execution environments, see the [documentation](https://docs.microsoft.com/en-gb/azure/machine-learning/service/how-to-deploy-and-where).

Deployment will take some time as it first runs a process to create a container image, and then runs a process to create a web service based on the image. When deployment has completed successfully, you'll see a status of **Healthy**.

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_diabetes.py",
                                   conda_file="diabetes_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Hopefully, the deployment has been successful and you can see a status of **Healthy**. If not, you can use the following code to check the status and get the service logs to help you troubleshoot.

In [ ]:
print(service.state)
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete the unhealthy service using the following code:
#service.delete()

Take a look at your workspace in the [Azure ML Studio web interface](https://ml.azure.com) and view the **Endpoints** page, which shows the deployed services in your workspace.

You can also enumerate the web services using the following code:

In [ ]:
for webservice_name in ws.webservices:
    webservice = ws.webservices[webservice_name]
    print(webservice.name)

With the service deployed, now you can consume it from a client application.

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

The code above uses the Azure ML SDK to connect to the containerized web service and use it to generate predictions from your diabetes classification model. In production, a model is likely to be consumed by business applications that do not use the Azure ML SDK, but simply make HTTP requests to the web service.

Let's determine the URL to which these applications must submit their requests:

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es). This time we'll send details for two patients, and get a prediction back for each.

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [5,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

In this notebook, you've seen the end-to-end process of ingesting data, training a model, and publishing the model as a real-time web service; all using the Azure ML SDK. For more details about the classes available in the SDK, take a look at the [Azure ML Python SDK reference](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py).